In [ ]:
import numpy as np
import math
from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{I} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [ ]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    # ... ENTER YOUR CODE HERE ...
    vec = np.asarray(vec, dtype=float) #vector tipo float
    norm_vec=np.linalg.norm(vec) #norma del vector usando np.linalg
    outvec = np.zeros_like(vec)  #construimos y con 0s
    outvec[0]=norm_vec #el primero elemento es igual a la norma de x
    norm = np.linalg.norm(vec-outvec) #norma de la resta x-y
    u = (vec - outvec) / norm #hallamos u con la formula
    I=np.eye(len(vec)) 
    H = I - 2*np.outer(u,u.T)# Hallamos H con la formula
    return  outvec, H
    

Test your function using tests below:

In [ ]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1,atol=1e-15)

In [ ]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

$$
\mathbf{A} = (\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1)^{-1} \mathbf{R} \;,
$$
so 
$$
\mathbf{A} =  \mathbf{Q} \mathbf{R} \;,
$$
with
$$
\mathbf{Q} = (\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1)^{-1} =  \mathbf{H}_1^{-1}  \mathbf{H}_2^{-1}  \cdots \mathbf{H}_{n-1}^{-1} = \mathbf{H}_1^T  \mathbf{H}_2^T  \cdots \mathbf{H}_{n-1}^T 
$$
Since $\mathbf{H}_i$ is ortogonal
$$
\mathbf{H}_i\mathbf{H}_i^T = \mathbf{I}
$$
then
$$
\mathbf{H}^{-1} = \mathbf{H}^T 
$$
but also  $\mathbf{H}_i$ is symetric
$$
\mathbf{H}_i^T = \mathbf{H}_i
$$
so
$$
\mathbf{Q} = \mathbf{H}_1^{-1}  \mathbf{H}_2^{-1}  \cdots \mathbf{H}_{n-1}^{-1} = \mathbf{H}_1^T  \mathbf{H}_2^T  \cdots \mathbf{H}_{n-1}^T =  \mathbf{H}_1 \mathbf{H}_2  \cdots \mathbf{H}_{n-1}
$$


In [ ]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    Q : ndarray, shape(m, m)
        The orthogonal matrix
    R : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    # ... ENTER YOUR CODE HERE ...
    (num_rows, num_cols) = np.shape(a)
    Q = np.eye(num_rows)
    R = np.copy(a)

    for i in range(num_cols-1): #por cada columna de la matriz, hallamos H usando la previamente elaborada
        x = R[i:, i] #x para los vectores de reflección

        Q_i = np.identity(num_rows)
        (y,Q_i[i:, i:]) = householder(x)  #Como householder retorna dos elementos, asignamos outvec a "y" y  lo ignoramos
        R = np.dot(Q_i, R)
        Q = np.dot(Q, Q_i.T)

    return Q,R

In [ ]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [ ]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [ ]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your q and r agree with qq and rr. Explain.

In [ ]:
from scipy.linalg import qr
a = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4], [9,3,1]])
q, r = qr_decomp(a)
qq, rr = qr(a)
print(q)
print(qq)
print(r)
print(rr)
print(a)
print(q@r)
print(qq@rr)

(5, 5)
(4, 4)
[[ 0.24479602  0.06723049  0.88467647  0.22294625  0.32124364]
 [ 0.30599503  0.58266423  0.18704939 -0.52598158 -0.50519831]
 [ 0.55079106  0.4964713  -0.40804552  0.44660369  0.29015419]
 [ 0.48959205 -0.47923271  0.03567661  0.36894623 -0.6270897 ]
 [ 0.55079106 -0.42406923 -0.12077289 -0.58143113  0.40514966]]
[[-0.24479602 -0.06723049  0.00981821 -0.41631113 -0.87300837]
 [-0.30599503 -0.58266423  0.33762629  0.64971343 -0.17535786]
 [-0.55079106 -0.4964713  -0.36457628 -0.41134781  0.38473703]
 [-0.48959205  0.47923271 -0.55563068  0.4547994  -0.1227505 ]
 [-0.55079106  0.42406923  0.66653641 -0.16884307  0.20979928]]
[[16.34013464 10.46503005  4.40632844]
 [-0.          6.51790964 -0.7843557 ]
 [-0.          0.         -0.31752655]
 [ 0.         -0.          2.45711113]
 [-0.         -0.         -0.91150293]]
[[-16.34013464 -10.46503005  -4.40632844]
 [  0.          -6.51790964   0.7843557 ]
 [  0.           0.          -2.63989693]
 [  0.           0.           0.

Enter your explanation here (10% of the total grade, peer-graded)

**La descomposición devuelve una matriz triangular r y una matriz perpendicular q**

**Tanto los de valores de q como de r coinciden con los valores de qq y de rr respectivamente. Sin embargo, las matrices obtenidas q y r están multiplicadas por -1, pero esto no es ningún problema pues la multiplicación q x r efectivamente coincide con A, comprobando que la descomposición de A en q y r fue exitosa.**


# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).





Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

$$
\mathbf{R} = \mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} 
$$
and 
$$
\mathbf{H}_i = \mathbf{I} - 2 \mathbf{u}_i \mathbf{u}_i^T
$$
so if
$$
\mathbf{R}_0 =  \mathbf{A}
$$
then 
$$
\mathbf{R}_1 = \mathbf{H}_1 \mathbf{R}_0= \ (\mathbf{I} - 2 \mathbf{u}_1 \mathbf{u}_1^T) \mathbf{R}_0 =  \mathbf{R}_0 -  2 \mathbf{u}_1 ( \mathbf{u}_1^T  \mathbf{R}_0)
$$
and
$$
\mathbf{R}_2 = \mathbf{H}_2 \mathbf{R}_1 = \ (\mathbf{I} - 2 \mathbf{u}_2 \mathbf{u}_2^T) \mathbf{R}_1 =  \mathbf{R}_1 -  2 \mathbf{u}_2 ( \mathbf{u}_2^T  \mathbf{R}_1)
$$
so on until
$$
\mathbf{R} = \mathbf{H}_n \mathbf{R}_{n-1} =  (\mathbf{I} - 2 \mathbf{u}_{n-1} \mathbf{u}_{n-1}^T) \mathbf{R}_{n-1} = \mathbf{R}_{n-1} -  2 \mathbf{u}_{n-1} ( \mathbf{u}_{n-1}^T  \mathbf{R}_{n-1})
$$

In [ ]:
def r_decomp(a):
    """Compute R without ever forming the  H  matrices
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    
    R : ndarray, shape(m, n)
        The upper triangular matrix
    vecs:  reflection vectors.
        
    """
    # ... ENTER YOUR CODE HERE ...
    (num_rows, num_cols) = np.shape(a)
    R = np.copy(a)
    R_i=0
    vecs= np.zeros((num_rows,num_cols))
    
    for i in range(num_cols): #por cada columna de la matriz hallamos cada R_i
        #print(i)
        #print("R\n",R)
        x = R[i:, i]
        norm_x=np.linalg.norm(x) #norma del vector usando np.linalg
        y = np.zeros_like(x)  #construimos y con 0s
        y[0]=norm_x #el primero elemento es igual a la norma de x
        norm = np.linalg.norm(x-y) #norma de la resta x-y
        u = (x - y) / norm #hallamos u con la formula
        R[i:,:] = R[i:,:] - 2.0*np.outer(u,np.dot(u.T,R[i:,:]))
              
        vec=np.zeros(num_rows)
       
        for j in range(len(u)):
          vec[j]=vec[j]+u[j]
        vecs[:,i]=vec
    #print("vecs",vecs)
    #print("R\n",R)
    return R, vecs

$$
\mathbf{Q} =  \mathbf{H}_1 \mathbf{H}_2  \cdots \mathbf{H}_{n-1}
$$
and
$$
\mathbf{H}_i = \mathbf{I} - 2 \mathbf{u}_i \mathbf{u}_i^T
$$
so 
$$
\mathbf{Q} =  (\mathbf{I} - 2 \mathbf{u}_1 \mathbf{u}_1^T )( \mathbf{I} - 2 \mathbf{u}_2 \mathbf{u}_2^T)  \cdots (\mathbf{I} - 2 \mathbf{u}_{n-1} \mathbf{u}_{n-1}^T)
$$

In [ ]:
def q_decomp(vecs):
    """Compute Q and QT from reflection vectors
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    
    Q : ndarray, shape(m, m)
        The ortogonal matrix
    Q.T : ndarray, shape(m, m)
        The transpose of Q

    """ 
    # ... ENTER YOUR CODE HERE ...
    (num_rows, num_cols) = np.shape(vecs)
    Q=np.eye(num_rows)
    for i in range(num_cols): #por cada columna de la matriz hallamos cada R_i
        u=vecs[:,i] #obtenemos cada vector de reflexión
        I=np.eye(len(u)-i)
        Q_i=np.eye(num_rows)
        Q_i[i:, i:]=I-2*(np.outer(u[:len(u)-i],u[:len(u)-i].T)) #hallamos el cada termino I-2uu'
        Q =np.dot(Q,Q_i.T)
        
    #print(Q)   
    return Q, Q.T

In [ ]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
r, vecs = r_decomp(a)
q,qt = q_decomp(vecs)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

In [ ]:
from scipy.linalg import qr
qq, rr = qr(a)
print(rr)
print(qq)
assert_allclose(np.dot(qq, rr), a)

[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]]


In [ ]:
a = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4], [9,3,1]])
R, vecs = r_decomp(a)
print("R\n ",R)
Q,QT = q_decomp(vecs)
print("Q\n",Q)
print("QT\n",QT)
print("QT@Q\n",QT@Q)
print("a\n",a)
print("Q@R\n",Q@R)

R
  [[16 10  4]
 [ 0  5  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]]
Q
 [[ 0.27088608  0.10705244  0.28933092  0.84195298  0.35009042]
 [ 0.30379747  0.57444244 -0.50150693  0.22061483 -0.52682339]
 [ 0.54683544  0.49113924  0.35443038 -0.48860759  0.30886076]
 [ 0.48607595 -0.45232068  0.42616034  0.01012658 -0.61434599]
 [ 0.54683544 -0.46124171 -0.59795057 -0.06003617  0.35647981]]
QT
 [[ 0.27088608  0.30379747  0.54683544  0.48607595  0.54683544]
 [ 0.10705244  0.57444244  0.49113924 -0.45232068 -0.46124171]
 [ 0.28933092 -0.50150693  0.35443038  0.42616034 -0.59795057]
 [ 0.84195298  0.22061483 -0.48860759  0.01012658 -0.06003617]
 [ 0.35009042 -0.52682339  0.30886076 -0.61434599  0.35647981]]
QT@Q
 [[ 1. -0. -0. -0. -0.]
 [-0.  1. -0. -0.  0.]
 [-0. -0.  1.  0.  0.]
 [-0. -0.  0.  1. -0.]
 [-0.  0.  0. -0.  1.]]
a
 [[4 3 1]
 [5 7 0]
 [9 9 3]
 [8 2 4]
 [9 3 1]]
Q@R
 [[4.33417722 3.24412297 1.0835443 ]
 [4.86075949 5.91018686 1.21518987]
 [8.74936709 7.92405063 2.18734177]
 [7.77721519